# TODO

* find the varid horizon

In [1]:
import gym, pickle, argparse, json, logging
from itertools import product
from copy import deepcopy
import tensorflow as tf
import ray

from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph
from ray.rllib.agents.ppo.ppo import DEFAULT_CONFIG
from ray.rllib.agents import Trainer
from ray.rllib.evaluation import PolicyEvaluator, SampleBatch
from ray.rllib.evaluation.metrics import collect_metrics
from ray.tune.registry import register_env
from ray.rllib.utils.annotations import override

from flow.multiagent_envs import MultiWaveAttenuationPOEnv
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder, get_flow_params

In [2]:
num_cpus = 3
num_rollouts = 3
horizon = 750
gae_lambda = 0.97
step_size = 5e-4
num_iter = 10
benchmark_name = "multi_merge"
exp_name = "test_ir"

In [3]:
ray.init(num_cpus=num_cpus, logging_level=40, ignore_reinit_error=True)

{'node_ip_address': '169.237.32.118',
 'object_store_address': '/tmp/ray/session_2019-05-25_07-23-08_10370/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-05-25_07-23-08_10370/sockets/raylet',
 'redis_address': '169.237.32.118:18870',
 'webui_url': None}

In [5]:
config = deepcopy(DEFAULT_CONFIG)
config["num_workers"] = min(num_cpus, num_rollouts)
config["train_batch_size"] = horizon * num_rollouts
config["sample_batch_size"] = horizon
config["use_gae"] = True
config["horizon"] = horizon
config["lambda"] = gae_lambda
config["lr"] = step_size
config["vf_clip_param"] = 1e6
config["num_sgd_iter"] = 10
config['clip_actions'] = False  # FIXME(ev) temporary ray bug
config["model"]["fcnet_hiddens"] = [128, 64, 32]
config["observation_filter"] = "NoFilter"
config["entropy_coeff"] = 0.0

benchmark = __import__(
            "flow.benchmarks.%s" % benchmark_name, fromlist=["flow_params"])
flow_params = benchmark.buffered_obs_flow_params

# save the flow params for replay
flow_json = json.dumps(
    flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
config['env_config']['flow_params'] = flow_json

In [6]:
create_env, env_name = make_create_env(params=flow_params, version=0)
register_env(env_name, create_env)
env = create_env()

default_policy = (PPOPolicyGraph, env.observation_space, env.action_space, {})
policy_graph = {"default_policy": default_policy}
config["multiagent"] = {
        'policy_graphs': policy_graph,
        'policy_mapping_fn': tune.function(lambda agent_id: "default_policy")
    }

In [7]:
class GailTrainer(Trainer):
    _name = "GAIL"
    _default_config = DEFAULT_CONFIG
    _policy_graph = PPOPolicyGraph
    
    @override(Trainer)
    def _init(self, config, env_creator):
        self.timestep = 0
        self.local_evaluator = self.make_local_evaluator(
             env_creator, self._policy_graph)        
        self.remote_evaluators = self.make_remote_evaluators(
            env_creator, self._policy_graph, config["num_workers"])
        
    @override(Trainer)    
    def _train(self):
        weights = ray.put(self.local_evaluator.get_weights())
        for e in self.remote_evaluators:
            e.set_weights.remote(weights)       
        batch = SampleBatch.concat_samples(
            ray.get([e.sample.remote() for e in self.remote_evaluators]))
        self.local_evaluator.learn_on_batch(batch)
        return collect_metrics(remote_evaluators=self.remote_evaluators)


In [8]:
agent = GailTrainer(config, env_name)

2019-05-25 07:23:16,295	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
/opt/conda/envs/flow-latest/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
2019-05-25 07:23:17,698	INFO policy_evaluator.py:728 -- Built policy map: {'default_policy': <ray.rllib.agents.ppo.ppo_policy_graph.PPOPolicyGraph object at 0x7f6f4d0a4630>}
2019-05-25 07:23:17,699	INFO policy_evaluator.py:729 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f6f4d0a4320>}
2019-05-25 07:23:17,700	INFO policy_evaluator.py:343 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f6f4d092630>}


In [9]:
agent._train()

(pid=10408) Loading configuration... done.
(pid=10408) Success.
(pid=10411) Loading configuration... done.
(pid=10411) Success.
(pid=10411) Loading configuration... done.
(pid=10408) Loading configuration... done.
(pid=10410) Loading configuration... done.
(pid=10410) Success.
(pid=10410) Loading configuration... done.
(pid=10411) 2019-05-25 07:23:29,055	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 1 on CPU (please ignore any CUDA init errors)
(pid=10411) 2019-05-25 07:23:29.056922: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
(pid=10408) 2019-05-25 07:23:29,039	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 3 on CPU (please ignore any CUDA init errors)
(pid=10408) 2019-05-25 07:23:29.041054: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F

(pid=10408) Loading configuration... done.
(pid=10408) Success.
(pid=10408) Loading configuration... done.
(pid=10410) Loading configuration... done.
(pid=10410) Success.
(pid=10410) Loading configuration... done.
(pid=10411) Loading configuration... done.
(pid=10411) Success.
(pid=10411) Loading configuration... done.
(pid=10408) Loading configuration... done.
(pid=10408) Success.
(pid=10408) Loading configuration... done.
(pid=10410) Loading configuration... done.
(pid=10410) Success.
(pid=10410) Loading configuration... done.
(pid=10411) 2019-05-25 07:23:41,664	INFO policy_evaluator.py:474 -- Completed sample batch:
(pid=10411) 
(pid=10411) { 'data': { 'action_prob': np.ndarray((1816,), dtype=float32, min=0.001, max=0.398, mean=0.282),
(pid=10411)             'actions': np.ndarray((1816, 1), dtype=float32, min=-3.029, max=3.49, mean=0.003),
(pid=10411)             'advantages': np.ndarray((1816,), dtype=float32, min=-23.125, max=-0.279, mean=-16.12),
(pid=10411)             'agent_i

2019-05-25 07:23:42,675	INFO policy_evaluator.py:564 -- Training on concatenated sample batches:

{ 'data': { 'action_prob': np.ndarray((5793,), dtype=float32, min=0.0, max=0.398, mean=0.279),
            'actions': np.ndarray((5793, 1), dtype=float32, min=-3.643, max=4.002, mean=0.002),
            'advantages': np.ndarray((5793,), dtype=float32, min=-23.125, max=-0.279, mean=-15.795),
            'agent_index': np.ndarray((5793,), dtype=int64, min=0.0, max=5.0, mean=2.313),
            'behaviour_logits': np.ndarray((5793, 2), dtype=float32, min=-0.003, max=0.007, mean=0.003),
            'dones': np.ndarray((5793,), dtype=bool, min=0.0, max=1.0, mean=0.006),
            'eps_id': np.ndarray((5793,), dtype=int64, min=42113611.0, max=1916260414.0, mean=639899390.127),
            'infos': np.ndarray((5793,), dtype=object, head={'mean_vel': 18.301188949654836, 'outflow': 445.5445544554455, 'cost1': 0.6573510345905751, 'cost2': 0.0}),
            'new_obs': np.ndarray((5793, 12), dtype=

{'custom_metrics': {},
 'episode_len_mean': 325.0,
 'episode_reward_max': -508.7997273337436,
 'episode_reward_mean': -622.2217628327572,
 'episode_reward_min': -785.6348404197764,
 'episodes_this_iter': 6,
 'num_metric_batches_dropped': 0,
 'off_policy_estimator': {},
 'policy_reward_mean': {},
 'sampler_perf': {'mean_env_wait_ms': 10.29710720974871,
  'mean_inference_ms': 0.9950023199048403,
  'mean_processing_ms': 4.377035157076264}}

In [7]:
tune.run_experiments({
    exp_name: {
        "run": GailTrainer,
        "env": env_name,
        "checkpoint_freq": 5,
        "max_failures": 999,
        "num_samples": 1,
        "stop": {
            "training_iteration": num_iter
        },
        "config": config
    }   
})

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 1.7/33.4 GB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 1.7/33.4 GB
Result logdir: /headless/ray_results/test_ir
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - GailTrainer_MultiWaveAttenuationMergePOEnvBufferedObs-v0_0:	RUNNING



2019-05-25 06:36:55,542	ERROR trial_runner.py:494 -- Error processing event.
Traceback (most recent call last):
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/trial_runner.py", line 443, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/ray_trial_executor.py", line 315, in fetch_result
    result = ray.get(trial_future[0])
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/worker.py", line 2193, in get
    raise value
ray.exceptions.RayTaskError: ray_GailTrainer:train() (pid=8953, host=kronos)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/rllib/agents/trainer.py", line 293, in __init__
    Trainable.__init__(self, config, logger_creator)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/trainable.py", line 88, in __init__
    self._setup(copy.deepcopy(self.config))
  File "/opt/conda/envs/flow-latest/lib/py

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 1.9/33.4 GB
Result logdir: /headless/ray_results/test_ir
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - GailTrainer_MultiWaveAttenuationMergePOEnvBufferedObs-v0_0:	RUNNING, 1 failures: /headless/ray_results/test_ir/GailTrainer_MultiWaveAttenuationMergePOEnvBufferedObs-v0_0_2019-05-25_06-36-45y51s7ud9/error_2019-05-25_06-36-55.txt



2019-05-25 06:37:05,623	ERROR trial_runner.py:494 -- Error processing event.
Traceback (most recent call last):
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/trial_runner.py", line 443, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/ray_trial_executor.py", line 315, in fetch_result
    result = ray.get(trial_future[0])
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/worker.py", line 2193, in get
    raise value
ray.exceptions.RayTaskError: ray_GailTrainer:train() (pid=8952, host=kronos)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/rllib/agents/trainer.py", line 293, in __init__
    Trainable.__init__(self, config, logger_creator)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/trainable.py", line 88, in __init__
    self._setup(copy.deepcopy(self.config))
  File "/opt/conda/envs/flow-latest/lib/py

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 1.9/33.4 GB
Result logdir: /headless/ray_results/test_ir
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - GailTrainer_MultiWaveAttenuationMergePOEnvBufferedObs-v0_0:	RUNNING, 2 failures: /headless/ray_results/test_ir/GailTrainer_MultiWaveAttenuationMergePOEnvBufferedObs-v0_0_2019-05-25_06-36-45y51s7ud9/error_2019-05-25_06-37-05.txt



2019-05-25 06:37:15,741	ERROR trial_runner.py:494 -- Error processing event.
Traceback (most recent call last):
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/trial_runner.py", line 443, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/ray_trial_executor.py", line 315, in fetch_result
    result = ray.get(trial_future[0])
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/worker.py", line 2193, in get
    raise value
ray.exceptions.RayTaskError: ray_GailTrainer:train() (pid=8949, host=kronos)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/rllib/agents/trainer.py", line 293, in __init__
    Trainable.__init__(self, config, logger_creator)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/trainable.py", line 88, in __init__
    self._setup(copy.deepcopy(self.config))
  File "/opt/conda/envs/flow-latest/lib/py

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 1.9/33.4 GB
Result logdir: /headless/ray_results/test_ir
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - GailTrainer_MultiWaveAttenuationMergePOEnvBufferedObs-v0_0:	RUNNING, 3 failures: /headless/ray_results/test_ir/GailTrainer_MultiWaveAttenuationMergePOEnvBufferedObs-v0_0_2019-05-25_06-36-45y51s7ud9/error_2019-05-25_06-37-15.txt



Exception in thread ray_print_logs:
Traceback (most recent call last):
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/redis/connection.py", line 177, in _read_from_socket
    raise socket.error(SERVER_CLOSED_CONNECTION_ERROR)
OSError: Connection closed by server.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/redis/client.py", line 2408, in _execute
    return command(*args)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/redis/connection.py", line 624, in read_response
    response = self._parser.read_response()
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/redis/connection.py", line 284, in read_response
    response = self._buffer.readline()
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/redis/connection.py", line 216, in readline
    self._read_from_socket()
  File "/opt/conda/envs/flow-latest/l

KeyboardInterrupt: 

In [ ]:
    _name = "GAIL"
    _default_config = DEFAULT_CONFIG
    _policy_graph = PPOPolicyGraph
    
    @override(Trainer)
    def _init(self, config, env_creator):
        tf.reset_default_graph()
        self.sess = tf.InteractiveSession()
        self.timestep = 0
        self.config = config
        
        # initialize policy and workers
        flow_params = get_flow_params(self.config)
        create_env, env_name = make_create_env(params=flow_params, version=0)
        env = create_env()
        default_policy = (self._policy_graph, env.observation_space, env.action_space, self.config)
        policy_graph = {"default_policy": default_policy}
        policy_mapping_fn = lambda agent_id: "default_policy"        
        
        self.policy = self._policy_graph(env.observation_space, env.action_space, self.config)    
        self.workers = [
            PolicyEvaluator.as_remote().remote(
                create_env, 
                policy_graph=policy_graph,
                policy_mapping_fn=policy_mapping_fn,
                batch_steps=self.config["sample_batch_size"],
                callbacks=self.config["callbacks"])
            for _ in range(self.config["num_workers"])]
        
        
    @override(Trainer)    
    def _train(self):
        weights = ray.put({"default_policy": self.policy.get_weights()})
        for w in self.workers:
            w.set_weights.remote(weights)       
        batch = SampleBatch.concat_samples(
            ray.get([w.sample.remote() for w in self.workers]))
        self.policy.learn_on_batch(batch)
        return collect_metrics(remote_evaluators=self.workers)